In [3]:
# sqlite and pandas required
# Jiyang Li 2025-10-08


import sqlite3
import pandas as pd
df = pd.read_csv('../data/PoliticalDiscussion_comments_sample.csv')
conn = sqlite3.connect('project_test.db')
df.to_sql('comment_table', conn, if_exists='replace', index=False)
post_number = pd.read_sql_query("select count(distinct parent_id) from comment_table ", conn)
print(post_number)



C:\Users\16082\AppData\Local\Temp\ipykernel_9192\717437559.py:7: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/PoliticalDiscussion_comments_sample.csv')


   count(distinct parent_id)
0                      48522


In [4]:
comment_number = pd.read_sql_query("select count(distinct comment_id) from comment_table ", conn)
print(comment_number)

   count(distinct comment_id)
0                       66942


In [5]:
# data structure
df.head(5)

,kind,subreddit,submission_id,comment_id,parent_id,is_top_level,body,author,score,created_utc,author_flair_text,author_flair_type,author_flair_template_id,author_flair_richtext
0,comment,PoliticalDiscussion,litsrx,gn9kh4p,t1_gn5o9dz,False,Pedantic but it does have kne exception. India...,Mist_Rising,27,1.613242e+09,NaN,text,NaN,[]
1,comment,PoliticalDiscussion,fjqvg5,fkp06ve,t3_fjqvg5,True,Tammy Baldwin or Elizabeth Warren are who I wa...,nantesorkestar,136,1.584399e+09,NaN,text,NaN,[]
2,comment,PoliticalDiscussion,w7yls1,ihngruw,t1_ihn2l7x,False,https://www.usatoday.com/story/news/politics/2...,lvlint67,6,1.658795e+09,NaN,text,NaN,[]
3,comment,PoliticalDiscussion,13zqss9,jmuzwk2,t1_jmuz8v0,False,"You bet, here:\n\nhttps://www.bbc.com/news/wor...",sbdude42,9,1.685885e+09,NaN,text,NaN,[]
4,comment,PoliticalDiscussion,13u8sip,jm0aig8,t1_jm09faz,False,I’ll tell you before I (involuntarily) lost my...,2057Champs__,6,1.685318e+09,NaN,text,NaN,[]


In [6]:
#should use pandas not sql :(
average_len = pd.read_sql_query("select sum(length(body))/count(*) from comment_table limit 5 ", conn)
print(average_len)

   sum(length(body))/count(*)
0                         368


In [7]:
# change created_utc to time
import pandas as pd
from datetime import datetime, timezone, timedelta


df['created_time'] = df['created_utc'].apply(
    lambda x: datetime.fromtimestamp(x, tz=timezone.utc)
)

print(df.head())


      kind            subreddit submission_id comment_id   parent_id  \
0  comment  PoliticalDiscussion        litsrx    gn9kh4p  t1_gn5o9dz   
1  comment  PoliticalDiscussion        fjqvg5    fkp06ve   t3_fjqvg5   
2  comment  PoliticalDiscussion        w7yls1    ihngruw  t1_ihn2l7x   
3  comment  PoliticalDiscussion       13zqss9    jmuzwk2  t1_jmuz8v0   
4  comment  PoliticalDiscussion       13u8sip    jm0aig8  t1_jm09faz   

   is_top_level                                               body  \
0         False  Pedantic but it does have kne exception. India...   
1          True  Tammy Baldwin or Elizabeth Warren are who I wa...   
2         False  https://www.usatoday.com/story/news/politics/2...   
3         False  You bet, here:\n\nhttps://www.bbc.com/news/wor...   
4         False  I’ll tell you before I (involuntarily) lost my...   

           author  score   created_utc author_flair_text author_flair_type  \
0     Mist_Rising     27  1.613242e+09               NaN            

In [8]:
# earliest and latest
earliest = df['created_time'].min()
latest = df['created_time'].max()
print("earliest time = ", earliest)
print("latest time = ", latest)

earliest time =  2012-01-25 15:55:43+00:00
latest time =  2023-12-30 13:28:46+00:00


In [9]:
# count by year
df['year'] = df['created_time'].dt.year
year_counts = df['year'].value_counts().sort_index()
print(year_counts)

year
2012       35
2013       58
2014       30
2015      117
2016     4992
2017     2226
2018     1509
2019     1296
2020     5906
2021     7627
2022    22189
2023    20957
Name: count, dtype: int64


In [ ]:
#check whether there are deleted comments
cursor = conn.cursor()
pattern = "%removed%" #avoid %r in text
cursor.execute("SELECT * FROM comment_table WHERE length(body)<=20 and body LIKE ?", (pattern,))
rows = cursor.fetchall()
print(rows)

[]


In [ ]:
cursor = conn.cursor()
pattern = "%deleted%" #avoid %r in text
cursor.execute("SELECT * FROM comment_table WHERE length(body)<=20 and body LIKE ?", (pattern,))
rows = cursor.fetchall()
print(rows)

[]


In [19]:
#merge submissions and comments
# Plz add the time transformation code to the merged data frame
# !!! data structure is a little bit messy here, try to clean it and drop some useless columns
import pandas as pd

comments = pd.read_csv("../data/PoliticalDiscussion_comments_sample.csv")
submissions = pd.read_csv("../data/PoliticalDiscussion_submissions_sample.csv")



C:\Users\16082\AppData\Local\Temp\ipykernel_20180\902415326.py:6: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  comments = pd.read_csv("../data/PoliticalDiscussion_comments_sample.csv")


In [17]:
comments["parent_clean_id"] = comments["parent_id"].str.replace(r"t\d+_", "", regex=True)
merged = pd.merge(
    submissions, comments,
    how="outer",
    left_on="submission_id",
    right_on="parent_clean_id",
    indicator=True
)

In [18]:
merged.head(10)

,kind_x,subreddit_x,submission_id_x,title,selftext,author_x,score_x,created_utc_x,link_flair_text,link_flair_type,...,body,author_y,score_y,created_utc_y,author_flair_text,author_flair_type,author_flair_template_id,author_flair_richtext,parent_clean_id,_merge
0,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,I'm pretty sure the only thing Trump knows how...,Djinnwrath,625.0,1.672521e+09,NaN,text,NaN,[],1003k02,both
1,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,"Well, it goes without saying that he's an atte...",infodawg,13.0,1.672521e+09,NaN,text,NaN,[],1003k02,both
2,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,He's not gonna run third party. He is trying t...,lightninhopkins,54.0,1.672521e+09,NaN,text,NaN,[],1003k02,both
3,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,Trump has already run as a Third Party Canidat...,LubbockGuy95,13.0,1.672522e+09,NaN,text,NaN,[],1003k02,both
4,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,Republicans cheered as Trump tore down every i...,SteveIDP,156.0,1.672521e+09,NaN,text,NaN,[],1003k02,both
5,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,"It's mostly attention seeking, and partly a th...",Wurm42,217.0,1.672521e+09,NaN,text,NaN,[],1003k02,both
6,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,Can we convince either Clinton or Obama to mak...,fletcherkildren,44.0,1.672521e+09,NaN,text,NaN,[],1003k02,both
7,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,Trump isn't saying anything. He is just [float...,OtakuOlga,24.0,1.672522e+09,NaN,text,NaN,[],1003k02,both
8,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,The title suggests that for Donald Trump there...,dave999dave,6.0,1.672522e+09,NaN,text,NaN,[],1003k02,both
9,submission,PoliticalDiscussion,1003k02,Donald Trump shared an article calling on him ...,Trump made sure his influence was felt during ...,PsychLegalMind,1012.0,1.672520e+09,US Politics,text,...,"If he did run third party, it would be the 3rd...",Catsandcamping,95.0,1.672521e+09,NaN,text,NaN,[],1003k02,both
